In [45]:
import pandas as pd
# import numpy as np
import re
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
# import tensorflow as tf
# from tensorflow import keras
# from keras import layers
# import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten, Conv1D
# from keras.utils import to_categorical
# from sklearn.metrics import accuracy_score
# import shutil

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [46]:
DETERMINATIVE_MAP = {
    # боги
    "ᴰ": "{d}",

    # звёзды
    "ᴹᵁᴸ": "{mul}",

    # земля / место
    "ᴷᴵ": "{ki}",

    # человек
    "ᴸᵁ₂": "{lu₂}",   # если lu₂ записано с надстрочной ₂
    "ᴸᵁ": "{lu₂}",    # частый OCR-вариант без ₂

    # здания
    "ᴱ₂": "{e₂}",
    "ᴱ": "{e₂}",

    # населённые пункты
    "ᵁᴿᵁ": "{uru}",

    # страны / горы
    "ᴷᵁᴿ": "{kur}",

    # женский род
    "ᴹᴵ": "{mi}",

    # мужской род
    "ᴹ": "{m}",

    # дерево / деревянное
    "ᴳᴵŠ": "{geš}",
    "ᴳᴵŠ": "{ĝeš}",   # при необходимости нормализации

    # ткани
    "ᵀᵁᴳ₂": "{tug₂}",
    "ᵀᵁᴳ": "{tug₂}",

    # таблички
    "ᴰᵁᴮ": "{dub}",

    # река / канал
    "ᴵᴰ₂": "{id₂}",
    "ᴵᴰ": "{id₂}",

    # птицы
    "ᴹᵁŠᴱᴺ": "{mušen}",
    "ᴹᵁŠ": "{mušen}",

    # камень
    "ᴺᴬ₄": "{na₄}",
    "ᴺᴬ": "{na₄}",

    # кожа
    "ᴷᵁŠ": "{kuš}",

    # растения
    "ᵁ₂": "{u₂}",
    "ᵁ": "{u₂}",
}
SUBSCRIPT_DIGITS = str.maketrans({
    "₀": "0",
    "₁": "1",
    "₂": "2",
    "₃": "3",
    "₄": "4",
    "₅": "5",
    "₆": "6",
    "₇": "7",
    "₈": "8",
    "₉": "9",
})

# --- ASCII → Unicode (фонетическая нормализация)
CHAR_MAP = {
    's"': 'š', 'S"': 'Š',
    's,': 'ṣ', 'S,': 'Ṣ',
    't,': 'ṭ', 'T,': 'Ṭ',
    'h,': 'ḫ', 'H,': 'Ḫ',
    "'": 'ʾ', "`": 'ʿ',
    "’": 'ʾ', "‘": 'ʿ',
}


In [47]:
def extract_quoted_substring(text: str, start_pos: int):
    """
    Ищет в строке text, начиная С позиции start_pos,
    подстроку вида: ' "текст"'.

    Возвращает:
        (substring, is_longer_than_30, closing_quote_pos)

    Если ничего не найдено — (None, None, None)
    """
    translate = False
    open_seq = ' "'

    # поиск начинается С start_pos
    open_pos = text.find(open_seq, start_pos)

    if open_pos == -1:
        return None, None, None

    # позиция открывающей кавычки "
    quote_start = open_pos + 1

    # ищем закрывающую кавычку "
    quote_end = text.find('"', quote_start + 1)

    if quote_end == -1:
        return None, None, None

    # подстрока между кавычками
    substring = text[quote_start + 1 : quote_end]

    if len(substring) > 30:
        translate = True
         # 1. найти открывающую скобку
        open_pos = text.find("(", quote_end)
        # if open_pos == -1:
        #     return None, None, None
        # 2. проверить расстояние от закрывающей кавычки до открывающей скобки
        if open_pos - quote_end > 3:
            translate = False

    return substring, translate, quote_end


In [48]:
def extract_parenthesized_substring(text: str, start_pos: int):
    """
    С позиции start_pos ищет '('.
    Возвращает:
        (substring, flag, close_pos)
    """

    # 1. найти открывающую скобку
    open_pos = text.find("(", start_pos)
    if open_pos == -1:
        return None, None, None

    # 2. проверить расстояние
    if open_pos - start_pos > 3:
        close_pos_tz = text.find(";", open_pos + 1)
        # скобка найдена, но слишком далеко
        close_pos_s = text.find(")", open_pos + 1)
        if close_pos_tz != -1 and close_pos_s != -1:
            close_pos = min(close_pos_tz, close_pos_s)
        else:
            if close_pos_tz == -1:
                close_pos = close_pos_s
            if close_pos_s == -1:
                close_pos = close_pos_tz
        if close_pos == -1:
            return None, None, None
        return text[open_pos + 1 : close_pos], False, close_pos

    # 3. найти закрывающую скобку
    close_pos_tz = text.find(";", open_pos + 1)
    # скобка найдена, но слишком далеко
    close_pos_s = text.find(")", open_pos + 1)
    if close_pos_tz != -1 and close_pos_s != -1:
        close_pos = min(close_pos_tz, close_pos_s)
    else:
        if close_pos_tz == -1:
            close_pos = close_pos_s
        if close_pos_s == -1:
            close_pos = close_pos_tz
    if close_pos == -1:
        return None, None, None

    substring = text[open_pos + 1 : close_pos]

    # 4. условия
    is_long = len(substring) > 30
    dash_count = substring.count("-")

    flag = is_long and dash_count >= 6

    return substring, flag, close_pos


In [49]:
def extract_letter_space_digit_colon_space(text: str, start_search_pos: int):
    # 1. Основной шаблон
    pattern = re.compile(
        r'[A-Za-z]{3,5} \d{4}: \d{2,}'
    )

    match = pattern.search(text, start_search_pos)
    if not match:
        return None, None, None

    # 2. Проверка 5 символов после группы
    pos = match.end()
    limit = min(pos + 5, len(text))

    start_pos = pos
    for i in range(pos, limit):
        if text[i].isdigit() or text[i] == '-':
            start_pos = i + 1
        else:
            start_pos = i
            break

    # 3. Поиск одинарной открывающей кавычки
    quote_pos = text.find("'", start_pos)
    if quote_pos == -1:
        return None, None, None

    diff = quote_pos - start_pos
    if diff >= 1000:
        return None, None, None

    # 4. Проверка подстроки
    substr = text[start_pos:quote_pos]

    dash_count = substr.count('-')
    aleph_count = substr.count('ℵ')

    dash_required = diff / 10.5

    if dash_count >= dash_required or aleph_count >= 2:
        transliter_txt = substr
        return transliter_txt, True, quote_pos - 1

    return None, None, None


In [50]:
def extract_single_quotes(text: str, start_pos: int):
    if start_pos < 0 or start_pos >= len(text):
        return None, None, None

    # 1. Поиск закрывающей одинарной кавычки
    quote_pos = text.find("'", start_pos)
    if quote_pos == -1:
        return None, None, None

    # 2. Проверка расстояния
    if quote_pos - start_pos > 1200:
        return None, None, None

    # 3. Извлечение подстроки
    translate_txt = text[start_pos:quote_pos]

    # 4. Возврат результата
    return translate_txt, True, quote_pos - 1


In [51]:
def normalize_akkadian_determinatives(text: str) -> str:
    for sup, norm in DETERMINATIVE_MAP.items():
        text = text.replace(sup, norm)
    return text


In [52]:
def normalize_subscripts(text: str) -> str:
    return text.translate(SUBSCRIPT_DIGITS)


In [53]:
def normalize_gaps(text: str) -> str:
    # порядок замен важен!
    replacements = [
        (r"\[\s*…\s*…\s*\]", "<big_gap>"),  # [… …]
        (r"\[x\]", "<gap>"),               # [x]
        (r"…", "<big_gap>"),               # …
    ]

    for pattern, repl in replacements:
        text = re.sub(pattern, repl, text)

    return text


In [54]:
# def normalize_assyrian_line(text: str) -> str:
#     # 1. Удаляем редакторскую метку Pl-/
#     text = re.sub(r"^Pl-/\s*", "", text)
#
#     # 2. Удаляем перенос строки (\)
#     text = text.replace("\\", "")
#
#     # 3. Удаляем запятую (маркер переноса строки)
#     text = text.replace(",", "")
#
#     # 4. Обрабатываем квадратные скобки
#     def replace_supplied(match):
#         content = match.group(1).strip()
#
#         # лакуны — не трогаем
#         if content == "x":
#             return "[x]"
#         if re.fullmatch(r"[.\s…]+", content):
#             return f"[{content}]"
#
#         # восстановленный текст → склеиваем
#         return "-" + content
#
#     text = re.sub(r"\[([^\]]+)\]", replace_supplied, text)
#
#     # 5. Убираем лишние пробелы
#     text = re.sub(r"\s+", " ", text).strip()
#
#     return text


In [55]:
def normalize_for_mt(text: str) -> str:
    # 0. Базовая очистка (translate-таблица уже применяется снаружи)
    a = text
    chars_to_remove = "!?/:.<>˹˺[]ℵ⅀⅁"
    table = str.maketrans("", "", chars_to_remove)
    # 1. ASCII → Unicode
    for old, new in CHAR_MAP.items():
        a = a.replace(old, new)

    # удаление ненужных символов
    a = a.translate(table)

    # 2. Надстрочные детерминативы
    a = normalize_akkadian_determinatives(a)

    # 3. Подстрочные цифры
    a = normalize_subscripts(a)

    # 4. Удаляем редакторские маркеры
    a = re.sub(r"^Pl-/\s*", "", a)   # Pl-/
    a = a.replace("\\", "")          # перенос строки
    a = a.replace(",", "")           # маркер переноса строки

    # 5. Квадратные скобки: восстановление vs лакуны
    def handle_brackets(match):
        content = match.group(1).strip()

        # лакуны
        if content == "x":
            return "<gap>"
        if re.fullmatch(r"[.…\s]+", content):
            return "<big_gap>"

        # восстановленный текст → включаем в слово
        return "-" + content

    a = re.sub(r"\[([^\]]+)\]", handle_brackets, a)

    # 6. Финальная нормализация пробелов
    a = re.sub(r"\s+", " ", a).strip()

    return a


In [56]:
def align_and_mark_sentences(translit_text: str, translation_sentences: list, marker="<sent>") -> str:
    """
    Точная выравнивающая функция для вставки маркеров конца предложений в транслитерацию.

    Args:
        translit_text: Нормализованная транслитерация (str)
        translation_sentences: Список английских предложений (list of str)
        marker: Спец-токен конца предложения (default "<sent>")

    Returns:
        Строка транслитерации с маркерами конца предложений
    """
    translit_tokens = translit_text.split()
    translation_lengths = [len(sent.split()) for sent in translation_sentences]
    total_translit = len(translit_tokens)
    total_translation = sum(translation_lengths)

    # Вычисляем пропорцию токенов транслитерации на токен перевода
    tokens_per_translation_token = total_translit / total_translation

    marked_tokens = []
    idx = 0

    for length in translation_lengths:
        # Сколько токенов транслитерации примерно для этого предложения
        num_tokens = max(1, round(length * tokens_per_translation_token))
        sent_tokens = translit_tokens[idx: idx + num_tokens]
        marked_tokens.extend(sent_tokens)
        marked_tokens.append(marker)
        idx += num_tokens

    # Добавляем остаток токенов, если есть
    if idx < total_translit:
        marked_tokens.extend(translit_tokens[idx:])
        marked_tokens.append(marker)

    return " ".join(marked_tokens)


In [57]:

def process_text_and_build_csv_rows(text: str):
    """
    Обрабатывает текст и возвращает список строк CSV
    (без заголовка)
    """
    csv_rows = []
    start_pos = 0

    while start_pos < len(text):
        # поиск по двойным кавычкам
        translate_str, flag, next_pos = extract_quoted_substring(text, start_pos)
        # if translate_str is None:
        #     break

        if flag:
            # print(translate_str)
            # поиск по круглым скобкам
            accad_str, flag2, close_pos = extract_parenthesized_substring(
                text, next_pos)

            if flag2:
                # 1. Очистка перевода
                t = translate_str.replace("\n", " ")

                # 2. Очистка аккадского
                a = accad_str.replace("\n", " ")
                a = normalize_for_mt(a)

                # 3. Токенизация перевода
                t_sentences = sent_tokenize(t)

                # 4. Выравнивание + маркеры
                a = align_and_mark_sentences(a, t_sentences, marker="<sent>")

                # 5. Склеиваем перевод обратно
                t = " ".join(t_sentences)

                # 6. CSV-экранирование (ОДИН РАЗ!)
                a = a.replace('"', '""')
                t = t.replace('"', '""')

                csv_rows.append(f'"{a}","{t}"\n')
                start_pos = close_pos + 1
            else:
                start_pos = next_pos + 1
        elif flag == False:
            start_pos = next_pos + 1
        else:
            # поиск по буквам пробел цифрам двоеточию пробелу цифрам
            accad_str, flag3, next_pos = extract_letter_space_digit_colon_space(text, start_pos)
            if flag3:
                # поиск по одинарным кавычкам
                translate_str, flag4, close_pos = extract_parenthesized_substring(text, next_pos)
                if flag4:
                    # 1. Очистка перевода
                    t = translate_str.replace("\n", " ")

                    # 2. Очистка аккадского
                    a = accad_str.replace("\n", " ")
                    a = normalize_for_mt(a)

                    # 3. Токенизация перевода
                    t_sentences = sent_tokenize(t)

                    # 4. Выравнивание + маркеры
                    a = align_and_mark_sentences(a, t_sentences, marker="<sent>")

                    # 5. Склеиваем перевод обратно
                    t = " ".join(t_sentences)

                    # 6. CSV-экранирование (ОДИН РАЗ!)
                    a = a.replace('"', '""')
                    t = t.replace('"', '""')

                    csv_rows.append(f'"{a}","{t}"\n')
                    start_pos = close_pos + 1
            else:
                start_pos = next_pos + 1
    return csv_rows



In [58]:
# ----------------------------
# Функция разбивки перевода на предложения
# ----------------------------
def naive_sent_tokenize(text):
    """
    Разделяет текст на предложения по точкам, восклицательным и вопросительным знакам.
    Работает для английского перевода.
    """
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return [s.strip() for s in sentences if s.strip()]

In [59]:
import csv
from io import StringIO

def parse_csv_line(line: str):
    reader = csv.reader(StringIO(line))
    accad_str, translate_str = next(reader)
    return accad_str, translate_str

In [60]:
# ----------------------------
# Выравнивание и разбивка транслитерации по <sent>
# ----------------------------
def split_accad_and_translate(csv_lines, marker="<sent>"):
    rows = []
    global_id = 0

    for line in csv_lines:
        accad_str, translate_str = parse_csv_line(line)

        accad_sentences = [s.strip() for s in accad_str.split(marker) if s.strip()]
        translate_sentences = naive_sent_tokenize(translate_str)

        min_len = min(len(accad_sentences), len(translate_sentences))
        accad_sentences = accad_sentences[:min_len]
        translate_sentences = translate_sentences[:min_len]

        for accad, trans in zip(accad_sentences, translate_sentences):
            rows.append({
                "id": global_id,
                "accad_str": accad,
                "translate": trans
            })
            global_id += 1

    return pd.DataFrame(rows, columns=["id", "accad_str", "translate"])


In [61]:
def print_file_head(path, n=5, encoding="utf-8"):
    with open(path, "r", encoding=encoding) as f:
        for i, line in enumerate(f):
            if i >= n:
                break
            print(f"{i}: {line.rstrip()}")


In [62]:
# Завантаження даних з CSV-файлу
# thiscompteca = "D:/Projects/Python/Конкурсы/Old_accad_translate/"
thiscompteca = "G:/Visual Studio 2010/Projects/Python/Old_accad_translate/"
csv_file_path = thiscompteca+'/data/publications.csv'
df_trnl = pd.read_csv(csv_file_path)

# print(df_trnl[df_trnl['has_akkadian']].head(20))  # Перші 5 строк даних
# print(df_trnl.shape)  # Dataset Shape
# print(df_trnl.info())  # Dataset Information
# print(df_trnl.describe())   # Statistics
# print(df_trnl.isnull().sum())  # Missing Values
print('\n')

# idx = df_trnl[df_trnl['has_akkadian']].head(3).index
idx = df_trnl[df_trnl['has_akkadian']].index
df_trnl.loc[idx, df_trnl.columns[2]] = (
    df_trnl.loc[idx, df_trnl.columns[2]]
    .str.replace("\\n", "\n", regex=False)
)
num = 0
all_rows = []
for i in idx:
    # print(f"index = {i}")
    # print("Назва файлу:", df_trnl.iat[i, 0])
    # print("Сторінка з текстом, що містить переклад:", df_trnl.iat[i, 1])
    # print("Текст всієї статті:\n", df_trnl.iat[i, 2])
    # print("-" * 50)
    list_row = []
    list_row = process_text_and_build_csv_rows(df_trnl.iat[i, 2])
    all_rows.extend(list_row)
    num += 1

# for i in idx[:10]:  # первые 10 для проверки
#     text = df_trnl.iat[i, 2]
#     rows = process_text_and_build_csv_rows(text)
#     print(f"Строка {i}: найдено {len(rows)} фрагментов")



new_df = split_accad_and_translate(all_rows)
new_df.to_csv('translate_from_publication.csv', index=False)
print("Примеры строк:")
print(new_df)
print(len(idx))
# print(num)
print('\n')


Traceback (most recent call last):
  File "G:\Program Files\JetBrains\PyCharm 2025.2\plugins\python-ce\helpers\pydev\_pydevd_bundle\pydevd_comm.py", line 736, in make_thread_stack_str
    append('file="%s" line="%s">' % (make_valid_xml_value(my_file), lineno))
                                     ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^
  File "G:\Program Files\JetBrains\PyCharm 2025.2\plugins\python-ce\helpers\pydev\_pydevd_bundle\pydevd_xml.py", line 36, in make_valid_xml_value
    return s.replace("&", "&amp;").replace('<', '&lt;').replace('>', '&gt;').replace('"', '&quot;')
           ^^^^^^^^^
AttributeError: 'tuple' object has no attribute 'replace'

KeyboardInterrupt



In [161]:
# Завантаження даних з CSV-файлу
# thiscompteca = "C:/Users/arecs/Мій диск (2armnot@gmail.com)/Питон/Конкурси/Old_Assyrian/"
csv_file_path = thiscompteca+'/data/published_texts.csv'
df_txt = pd.read_csv(csv_file_path)
num_row = 0
for num_row in range(df_txt.shape[0]):
    if num_row > 3:
        break
    for num_col in range(df_txt.shape[1]):
        print(df_txt.iat[num_row, num_col])
    print('-' * 50)

# print(df_txt.head())  # Перші 5 строк даних
# print(df_txt.shape)  # Dataset Shape
# print(df_txt.info())  # Dataset Information
# print(df_txt.describe())   # Statistics
# print(df_txt.isnull().sum())  # Missing Values

bd4b7138-70d5-490d-8234-299b0a75d3a3
https://oare.byu.edu/epigraphies/bd4b7138-70d5-490d-8234-299b0a75d3a3
P361099
MP 1 bis
Cuneiform Tablet RA 60 125
RA 60 125
Cuneiform envelope, dated to the Old Assyrian Period (ca. 1950-1850 BC).
letter
MP 1 bis
nan
Collection: MP 1 bis
P. Garelli RA 60 S. 124-125; Michel Innaya II, 214 #10 Hüllenfrgt.
nan
nan
4395
nan
https://aicuneiform.com/search?q=P361099
KIŠIB a-šùr-ma-lik DUMU i-na-a a-na a-šùr-na-da DUMU … ù en-um-a-šùr DUMU … a-pu-tum a-na a-wa-at ṭup-pì-im iḫ-da … ša ú-… ḫa-sí-sú …
KIŠIB a-šùr-ma-lik DUMU i-na-a a-na a-šùr-na-da DUMU <big_gap> ù en-um-a-šùr DUMU <big_gap> a-pu-tum a-na a-wa-at ṭup-pì-im iḫ-da <big_gap> ša ú-<big_gap> ḫa-sí-sú <big_gap>
--------------------------------------------------
b05376c2-fc3d-49f8-9792-a25c0df9c383
https://oare.byu.edu/epigraphies/b05376c2-fc3d-49f8-9792-a25c0df9c383
P359543
ICK 1 146
Cuneiform Tablet ICK 1 146
Ka 185 | Ka 455 | Ka 424?
Cuneiform tablet, dated to the Old Assyrian Period (ca. 1950-18

In [4]:
# Завантаження даних з CSV-файлу
# thiscompteca = "C:/Users/arecs/Мій диск (2armnot@gmail.com)/Питон/Конкурси/Old_Assyrian/"
csv_file_path = thiscompteca+'/data/bibliography.csv'
df_txt = pd.read_csv(csv_file_path)


# print(df_txt.head())  # Перші 5 строк даних
# print(df_txt.shape)  # Dataset Shape
print(df_txt.info())  # Dataset Information
# print(df_txt.describe())   # Statistics
# print(df_txt.isnull().sum())  # Missing Values

num_row = 0
for num_row in range(df_txt.shape[0]):
    # if num_row > 10:
    #     break
    for num_col in range(df_txt.shape[1]):
        if df_txt.iat[num_row, 2] == 'Mogens Trolle Larsen':
            print(df_txt.iat[num_row, num_col])
            if num_col == df_txt.shape[1] - 1:
                print('-' * 50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pdf_name      878 non-null    object
 1   title         871 non-null    object
 2   author        846 non-null    object
 3   author_place  544 non-null    object
 4   journal       583 non-null    object
 5   volume        568 non-null    object
 6   year          878 non-null    object
 7   pages         533 non-null    object
dtypes: object(8)
memory usage: 55.0+ KB
None
(OCR) JCS 59, 2007, pp. 93-106 - Larsen, Mogens T. - Individual and Family in Old Assyrian Society.pdf
Individual and Family in Old Assyrian Society
Mogens Trolle Larsen
Copenhagen
JCS
59
2007
93–95
--------------------------------------------------
AKT 6a.pdf
The Archive of the Salim-Assur Family
Mogens Trolle Larsen
Ankara
Turk Tarih Kurumu Yayınları
VI. Dizi- Sayı 33 d-a
2010
xvi, 585 s.
------------------------------------

In [5]:
# Завантаження даних з CSV-файлу
# thiscompteca = "C:/Users/arecs/Мій диск (2armnot@gmail.com)/Питон/Конкурси/Old_Assyrian/"
csv_file_path = thiscompteca+'/data/train.csv'
df_txt = pd.read_csv(csv_file_path)
num_row = 0
for num_row in range(df_txt.shape[0]):
    if num_row > 5:
        break
    for num_col in range(df_txt.shape[1]):
        print(df_txt.iat[num_row, num_col])
    print('-' * 50)

# print(df_txt.head())  # Перші 5 строк даних
# print(df_txt.shape)  # Dataset Shape
# print(df_txt.info())  # Dataset Information
# print(df_txt.describe())   # Statistics
# print(df_txt.isnull().sum())  # Missing Values

004a7dbd-57ce-46f8-9691-409be61c676e
KIŠIB ma-nu-ba-lúm-a-šur DUMU ṣí-lá-(d)IM KIŠIB šu-(d)EN.LÍL DUMU ma-nu-ki-a-šur KIŠIB MAN-a-šur DUMU a-ta-a 0.33333 ma-na 2 GÍN KÙ.BABBAR SIG₅ i-ṣé-er PUZUR₄-a-šur DUMU a-ta-a a-lá-ḫu-um i-šu iš-tù ḫa-muš-tim ša ì-lí-dan ITU.KAM ša ke-na-tim li-mu-um e-na-sú-in a-na ITU 14 ḫa-am-ša-tim i-ša-qal šu-ma lá iš-qú-ul 1.5 GÍN.TA a-na 1 ma-na-im i-na ITU.1.KAM ṣí-ib-tám ú-ṣa-áb
Seal of Mannum-balum-Aššur son of Ṣilli-Adad, seal of Šu-Illil son of Mannum-kī-Aššur, seal of Puzur-Aššur son of Ataya. Puzur-Aššur son of Ataya owes 22 shekels of good silver to Ali-ahum. Reckoned from the week of Ilī-dan, month of Ša-kēnātim, in the eponymy of Enna-Suen, he will pay in 14 weeks. If he has not paid in time, he will add interest at the rate 1.5 shekel per mina per month.
--------------------------------------------------
0064939c-59b9-4448-a63d-34612af0a1b5
1 TÚG ša qá-tim i-tur₄-DINGIR il₅-qé
Itūr-ilī has received one textile of ordinary quality.
----------------